## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
# File location and type
file_location = "/FileStore/tables/tips.csv"
file_type = "csv"

pyspark = spark.read.csv(file_location, header = True, inferSchema = True)
pyspark.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

In [0]:
pyspark.printSchema()

root
 |-- total_bill: double (nullable = true)
 |-- tip: double (nullable = true)
 |-- sex: string (nullable = true)
 |-- smoker: string (nullable = true)
 |-- day: string (nullable = true)
 |-- time: string (nullable = true)
 |-- size: integer (nullable = true)



In [0]:
pyspark.columns

Out[13]: ['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size']

In [0]:
# Handling Categorical Features

from pyspark.ml.feature import StringIndexer

In [0]:
indexer = StringIndexer(inputCol = 'sex',outputCol = 'sex_indexed')
df_pyspark = indexer.fit(pyspark).transform(pyspark)
df_pyspark.show()

+----------+----+------+------+---+------+----+-----------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|
+----------+----+------+------+---+------+----+-----------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|        0.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|        0.0|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|        0.0|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|        0.0|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|        0.0|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|        0.0|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|        1.0|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|        0.0|
|     18.43| 3.0|  Male|    No|Sun|Dinne

In [0]:
indexer = StringIndexer(inputCols = ['smoker','day','time'],outputCols = ['smoker_indexed','day_indexed','time_indexed'])
df_pyspark = indexer.fit(df_pyspark).transform(df_pyspark)
df_pyspark.show()

+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|smoker_indexed|day_indexed|time_indexed|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|           0.0|        1.0|         0.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|           0.0|        1.0|         0.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|        0.0|           0.0|        1.0|         0.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|


In [0]:
df_pyspark.columns

Out[17]: ['total_bill',
 'tip',
 'sex',
 'smoker',
 'day',
 'time',
 'size',
 'sex_indexed',
 'smoker_indexed',
 'day_indexed',
 'time_indexed']

In [0]:
from pyspark.ml.feature import VectorAssembler

In [0]:
feature_assemble = VectorAssembler(inputCols = ['tip','size','sex_indexed','smoker_indexed','day_indexed','time_indexed'], outputCol = 'independant_variables')
output = feature_assemble.transform(df_pyspark)
output.show()

+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+---------------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|smoker_indexed|day_indexed|time_indexed|independant_variables|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+---------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|           0.0|        1.0|         0.0| [1.01,2.0,1.0,0.0...|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0| [1.66,3.0,0.0,0.0...|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0| [3.5,3.0,0.0,0.0,...|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0| [3.31,2.0,0.0,0.0...|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|           0.0|        1.0|         0.0| [3.61,4.0,1.0,0.0...|
|     25.29|4.71|  Male|

In [0]:
final_data = output.select('independant_variables','total_bill')
final_data.show()

+---------------------+----------+
|independant_variables|total_bill|
+---------------------+----------+
| [1.01,2.0,1.0,0.0...|     16.99|
| [1.66,3.0,0.0,0.0...|     10.34|
| [3.5,3.0,0.0,0.0,...|     21.01|
| [3.31,2.0,0.0,0.0...|     23.68|
| [3.61,4.0,1.0,0.0...|     24.59|
| [4.71,4.0,0.0,0.0...|     25.29|
| [2.0,2.0,0.0,0.0,...|      8.77|
| [3.12,4.0,0.0,0.0...|     26.88|
| [1.96,2.0,0.0,0.0...|     15.04|
| [3.23,2.0,0.0,0.0...|     14.78|
| [1.71,2.0,0.0,0.0...|     10.27|
| [5.0,4.0,1.0,0.0,...|     35.26|
| [1.57,2.0,0.0,0.0...|     15.42|
| [3.0,4.0,0.0,0.0,...|     18.43|
| [3.02,2.0,1.0,0.0...|     14.83|
| [3.92,2.0,0.0,0.0...|     21.58|
| [1.67,3.0,1.0,0.0...|     10.33|
| [3.71,3.0,0.0,0.0...|     16.29|
| [3.5,3.0,1.0,0.0,...|     16.97|
| (6,[0,1],[3.35,3.0])|     20.65|
+---------------------+----------+
only showing top 20 rows



In [0]:
from pyspark.ml.regression import LinearRegression
train_data,test_data = final_data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol = 'independant_variables', labelCol = 'total_bill')
regressor = regressor.fit(train_data)

In [0]:
regressor.coefficients

Out[24]: DenseVector([2.9659, 3.5884, -0.9644, 2.6536, -0.2997, -0.7433])

In [0]:
regressor.intercept

Out[26]: 1.5981448264826783

In [0]:
pred_results = regressor.evaluate(test_data)

In [0]:
pred_results.predictions.show()

/databricks/spark/python/pyspark/sql/context.py:134: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(
+---------------------+----------+------------------+
|independant_variables|total_bill|        prediction|
+---------------------+----------+------------------+
|  (6,[0,1],[2.0,2.0])|     12.69|14.706585729942649|
| (6,[0,1],[2.24,3.0])|     16.04|19.006751369506947|
|  (6,[0,1],[2.5,4.0])|     18.35| 23.36623424097034|
| (6,[0,1],[3.15,3.0])|     20.08|21.705685420915867|
| (6,[0,1],[3.18,2.0])|     19.82| 18.20630241198938|
| (6,[0,1],[3.39,2.0])|     11.61|  18.8291333469299|
| (6,[0,1],[6.73,4.0])|     48.27| 35.91182878762938|
| [1.44,2.0,0.0,0.0...|      7.56|11.702859338706038|
| [1.5,2.0,0.0,0.0,...|     12.46|12.324413139615402|
| [1.5,2.0,0.0,1.0,...|     12.03|14.977989896540961|
| [1.5,2.0,1.0,0.0,...|     10.65|10.916420232313474|
| [1.5,2.0,1.0,0.0,...|     11.17|10.916420232313474|
| [1.56,2.0,0.0,0.0...|      9.94|13

In [0]:
pred_results.r2

Out[32]: 0.5045938952611131